<a href="https://colab.research.google.com/github/jade-smith/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# "Prediction of Product Sales"


In [ ]:
- "Author: Jade Smith"

In [ ]:
## "Project Overview"


In [ ]:
## "Load and Inspect Data"


In [ ]:
## "Clean Data"


In [ ]:
## "Exploratory Data Analysis"


In [ ]:
## "Feature Inspection"
